In [1]:
import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt
import io
import csv
%matplotlib qt5

In [2]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = 9000000
T_files = samplesPerFile / 10e6

In [3]:
#EKF NAV
idx_timeStamp = 0
idx_velNorth = 1
idx_velEast = 2
idx_velDown = 3
idx_velNorth_StdDev = 4
idx_velEast_StdDev = 5
idx_velDown_StdDev = 6
idx_Lat = 7
idx_Long = 8
idx_Alt = 9
idx_undulation = 10
idx_Lat_StdDev = 11
idx_Long_StdDev = 12
idx_Alt_StdDev = 13
idx_nav_status = 14

Lat_1_0  = 48.06069 
Long_1_0 = -1.99354
Lat_1_1  = 48.05507 
Long_1_1 = -2.02359

Lat_2_0  = 48.06249 
Long_2_0 = -1.99467
Lat_2_1  = 48.05687 
Long_2_1 = -2.02434

Lat_3_0  = 48.06555 
Long_3_0 = -1.99619
Lat_3_1  = 48.06007 
Long_3_1 = -2.02550

runaway = np.array([ [ 48.057546, -2.010483, 0.0 ],
          [ 48.058403, -2.005964, 0.0 ],
          [ 48.058191, -2.005869, 0.0 ],
          [ 48.057327, -2.010383, 0.0 ],
          [ 48.057546, -2.010483, 0.0 ]
          ])

In [4]:
prefix = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-2/2017_09_21-22_SBG/dataLogger"

logEkfNav  = prefix + "/session_0001/2017_09_22/09h00/" + "sbgLogEkfNav.dat"
logUtcData = prefix + "/session_0001/2017_09_22/09h00/" + "sbgLogUtcData.dat"

logEkfNav_2  = prefix + "/session_0001/2017_09_22/10h00/" + "sbgLogEkfNav.dat"
logUtcData_2 = prefix + "/session_0001/2017_09_22/10h00/" + "sbgLogUtcData.dat"

## Load navigation data SbgLogEkfNav

In [5]:
# logEkfNav
timeStamps = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_timeStamp )
Long = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Long )
Lat = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Lat )
Alt = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Alt )
vN, vE, vD = np.loadtxt( logEkfNav, skiprows = 1, usecols = (idx_velNorth, idx_velEast, idx_velDown), unpack=True )

In [6]:
Vel = (vN**2 + vE**2 + vD**2)**0.5

In [7]:
# logEkfNav
timeStamps_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_timeStamp )
Long_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_Long )
Lat_2 = np.loadtxt( logEkfNav_2, skiprows = 1, usecols = idx_Lat )

### Plot navigation data jointly with track references

In [8]:
idx_track2_0 = 432379
idx_track2_1 = 444144

In [9]:
plt.figure()

plt.plot( Long, Lat, 'grey' )
plt.plot( Long_2, Lat_2, 'black' )

plt.plot( [Long_1_0, Long_1_1], [Lat_1_0, Lat_1_1], "o-b" )
plt.plot( [Long_2_0, Long_2_1], [Lat_2_0, Lat_2_1], "o-b" )
plt.plot( [Long_3_0, Long_3_1], [Lat_3_0, Lat_3_1], "o-b" )

plt.plot( Long[idx_track2_0], Lat[idx_track2_0], 'or' )
plt.plot( Long[idx_track2_1], Lat[idx_track2_1], 'or' )

plt.title("Navigation data and track references 2017_09_22 09h00-10h00")

In [10]:
plt.figure()
ax1 = plt.subplot(211)
ax1.plot(Long)
ax1.plot(np.ones(Long.shape) * Long_2_0)
ax1.plot(np.ones(Long.shape) * Long_2_1)
ax2 = plt.subplot(212, sharex=ax1)
ax2.plot(Lat)
ax2.plot(np.ones(Lat.shape) * Lat_2_0)
ax2.plot(np.ones(Lat.shape) * Lat_2_1)

### Extract UTC data corresponding to the track 2

In [29]:
def getDataAtIndex(filename, index):
    with open(filename, newline='') as f:
        reader = csv.reader(f)
        for r in range(index+1):
            next(reader)
        val = next(reader)
    return val

In [31]:
getDataAtIndex(logUtcData, idx_track2_0)

['1366127704 167 2017 9 22 9 53 10 245001000 467608245 ']

In [32]:
getDataAtIndex(logUtcData, idx_track2_1)

['1424952704 167 2017 9 22 9 54 9 70001000 467667070 ']

### Extract UTC data corresponding to the date of the recorded files

In [ ]:
hourOffset = 2

def getUtcData( logUtcData, timeOfDay ):
    with open( logUtcData, newline='') as f:
        reader = csv.reader(f)
        next(reader)
        utc = np.loadtxt( io.StringIO( next(reader)[0] ))
        
        while (utc[5] < timeOfDay[0] - hourOffset):
            utc = np.loadtxt( io.StringIO( next(reader)[0] ))
            
        while (utc[6] < timeOfDay[1]):
            utc = np.loadtxt( io.StringIO( next(reader)[0] ))
            
        while (utc[7] < timeOfDay[2]):
            utc = np.loadtxt( io.StringIO( next(reader)[0] ))
            
    return utc

In [ ]:
firstStoredBlock = 12
t_0    = (11, 43, 45)
t_12   = (11, 43, 48)
t_18   = (11, 43, 49)
t_7734 = (12,  3,  6)

In [ ]:
utc_startRecord = getUtcData( logUtcData, t_12 )

In [ ]:
idx_startRecord = int(np.where(timeStamps == utc_startRecord[0])[0])

In [ ]:
utc_stopRecord = getUtcData( logUtcData_2, t_7734 )

In [ ]:
idx_stopRecord = int(np.where(timeStamps_2 == utc_stopRecord[0])[0])

In [ ]:
plt.figure()

# plot longitude/latitude and track references

plt.plot( Long, Lat, 'gray' )
plt.plot( Long_2, Lat_2, 'black' )
plt.plot( [Long_1_0, Long_1_1], [Lat_1_0, Lat_1_1], "o-b" )
plt.plot( [Long_2_0, Long_2_1], [Lat_2_0, Lat_2_1], "o-b" )
plt.plot( [Long_3_0, Long_3_1], [Lat_3_0, Lat_3_1], "o-b" )

# add on the plot the track during which data are recorded

plt.plot( Long[idx_startRecord], Lat[idx_startRecord], 'D', color='orange' )
plt.plot( Long_2[idx_stopRecord], Lat_2[idx_stopRecord], 'D', color='red' )

plt.plot( Long[idx_startRecord:], Lat[idx_startRecord:], 'orange' )
plt.plot( Long_2[:idx_stopRecord], Lat_2[:idx_stopRecord], 'red' )

plt.plot(runaway[:,1], runaway[:,0], "og")

plt.title("Navigation data and track references 2017_09_22 09h00-10h00")

### Select data for focalization

In [ ]:
def getTimeOfDay( seconds ):
    h = np.floor( seconds / 3600 )
    m = np.floor( (seconds - h *3600) / 60 )
    s = seconds - h * 3600 - m * 60
    return (h, m, s)

def getSeconds( timeOfDay ):
    return (timeOfDay[0] * 3600 + timeOfDay[1] * 60 + timeOfDay[2])

def getBlockNumber( t, t_0 ):
    delta = getSeconds( t ) - getSeconds(t_0)
    return np.floor( delta / T_files * blocksPerFile )

In [ ]:
tod = getTimeOfDay( getSeconds(t_18) )
utc_val = getTimeCode( logUtcData, tod)
idx_startOfSelection = int(np.where(timeStamps == utc_val[0])[0])

In [ ]:
# 11 files
numberOfFilesToKeep = 10
endOfSelection_t = getTimeOfDay( getSeconds(t_18) + numberOfFilesToKeep * T_files )
utc_endOfSelection = getTimeCode( logUtcData, endOfSelection_t )
idx_endOfSelection = int(np.where(timeStamps == utc_endOfSelection[0])[0])

In [ ]:
fig = plt.figure()

# plot longitude/latitude and track references

plt.plot( Long, Lat, 'gray' )
plt.plot( Long_2, Lat_2, 'black' )
plt.plot( [Long_1_0, Long_1_1], [Lat_1_0, Lat_1_1], "o-.b" )
plt.plot( [Long_2_0, Long_2_1], [Lat_2_0, Lat_2_1], "o-.b" )
plt.plot( [Long_3_0, Long_3_1], [Lat_3_0, Lat_3_1], "o-.b" )

plt.plot(runaway[:,1], runaway[:,0], "og")

# add on the plot the track during which data are recorded

plt.plot( Long[idx_startOfSelection], Lat[idx_startOfSelection], 'D', color='orange' )
plt.plot( Long[idx_endOfSelection], Lat[idx_endOfSelection], 'D', color='orange' )
plt.plot( Long[idx_startOfSelection:idx_endOfSelection], Lat[idx_startOfSelection:idx_endOfSelection], color='orange' )

plt.title("Navigation data and track references 2017_09_22 09h00-10h00\n Data selection")

In [ ]:
altitude = np.average(Alt[idx_startOfSelection:idx_endOfSelection])
velocity = np.average(Vel[idx_startOfSelection:idx_endOfSelection])

In [ ]:
altitude

In [ ]:
velocity

In [ ]:
altitude*2**0.5

In [ ]:
velocity * 3600 / 1000

# Annexe

### Check the dates of the recorded files

In [ ]:
def getLastLine( filename ):
    with open(filename, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            last = next(reader)
        print(last)

In [ ]:
getSeconds(t_0) + 3372 / 6 * T_files

In [ ]:
getTimeOfDay( 55816.8 )

In [ ]:
getBlockNumber( (13 + hourOffset, 23, 50), t_0)